# C2Score Workbench in JavaScritp/TypeScript.

In [12]:
// https://nbviewer.org/github/dbuezas/tslab-plotly/blob/master/example/notebook.ipynb
import Plotly from 'tslab-plotly';
import * as tslab from 'tslab';
import { C2Utils } from './c2utils';
import * as mysql from 'mysql';

In [13]:
// Connect to MySql
const connection = mysql.createConnection({host: '10.0.0.100',user: 'c2public',password: 'c2public',database: 'c2score'});
connection.connect((err) => {
    if(err) throw err;
    console.log('Connected to MySQL Server!');
});

Connected to MySQL Server!


In [14]:
// We can select just needed dtabase columns or the whole data:
const sql = "SELECT * from c2score_ScoringWorkbenchView;"
let resultArray = [];

const q = connection.query(sql, (err, rows, fields) => {
        if(err) throw err;
        resultArray = Object.values(JSON.parse(JSON.stringify(rows)));
        console.log("Data ready");
        connection.end();
    });


Data ready


In [15]:
// The first row in the resultArray:
resultArray[0];

{
  StrategyId: 13202557,
  StrategyName: 'extreme-os',
  Added: '2005-02-17T00:00:00.000Z',
  StartingCash: 10000,
  AgeDays: 6091.7,
  Alpha: 0.0628,
  Beta: 0.5596,
  C2Star: null,
  AnnReturn: 0.2826,
  Correlation_SP500: 0.3792,
  DailyMaxLevMax: 2.72,
  DailyMaxLevMean: 0.839638,
  DeltaEquity30Days: 5.64,
  DeltaEquity45Days: 5.16,
  DeltaEquity90Days: 15.89,
  DeltaEquity180Days: 6.72,
  MaxDrawdownPcnt: 63.64,
  NumTrades: 4206,
  PcntMonthsProfitable: 70.1,
  ProfitFactor: 1.366,
  Sharpe: 0.79,
  Sortino: 1.19,
  Calmar: 0.198,
  MaxDrawdownPcnt180days: 17.4534,
  MaxDrawdownPcnt365days: 37.7282,
  TradeDaysAgo: 0.7,
  ShortOptionsCovered: 100,
  OptionPercent: 0,
  MaxWorstLossPercentEquity015: 5.13,
  MaxWorstLossPercentEquity030: 6.63,
  MaxWorstLossPercentEquity045: 13.3,
  MaxWorstLossPercentEquity060: 13.3,
  MaxWorstLossPercentEquity090: 28.68,
  MaxWorstLossPercentEquity180: 36.54
}


In [16]:
// Let's implement Daniil's formula in TypeScript. (It can be done by many different ways.) 

// Risk measure
function riskFun(strat) {
    const result = 
      (20 - strat.MaxDrawdownPcnt * (20/45))
    + (strat.MaxWorstLossPercentEquity060 === null ? 0 : (20 - strat.MaxWorstLossPercentEquity060 * (20.0/8)))
    + (10 - strat.OptionPercent * (10/1))
    + (strat.ShortOptionsCovered >= 100 ? 10 : strat.ShortOptionsCovered / 100 * 10);
    return result;
}

// Behavior measure
function behavioralFun(strat) {
    const result = 
      (strat.AgeDays >= 720 ? 40 : strat.AgeDays/720 * 40)
    + (20 - strat.DailyMaxLevMax * (20/10)) 
    + (strat.TradeDaysAgo <= 60 ? 20 : 0)
    + (strat.PcntMonthsProfitable >= 100 ? 5 : strat.PcntMonthsProfitable / 100 * 5)
    + (strat.NumTrades >= 100 ? 20 : strat.NumTrades / 100 * 20)
    + ((strat.DeltaEquity90Days <= 0 && strat.DeltaEquity90Days > -7) ? 5 : 0);
    return result;
}

// Profitability measure
function profitabilityFun(strat) {
    const result = strat.AnnReturn > 0.6 ? 20 :  strat.AnnReturn / 0.6 * 20;
    return result;
}

// Filter data by Daniil's conditions and apply the above functions.
// Add new colums to the original data in resultArray.
let data = resultArray
            .filter(strat => !(strat.AgeDays < 90 || strat.AnnReturn < 0 || strat.DeltaEquity180Days < 0))
            .map(strat => {
                return {...strat, 
                        ...{scoreRisk:riskFun(strat)},
                        ...{scoreBehavior: behavioralFun(strat)},
                        ...{scoreProfitability: profitabilityFun(strat)},
                        ...{score: (riskFun(strat) + behavioralFun(strat) + profitabilityFun(strat))}
                       }})
            .sort((a,b) => b.score - a.score );

console.log(`Strategies: ${resultArray.length}. After filtering: ${data.length}`);

Strategies: 727. After filtering: 289


In [17]:
// What we have now:
data[0];

{
  StrategyId: 128743489,
  StrategyName: 'JWB Stock and ETF ORB',
  Added: '2020-04-26T00:00:00.000Z',
  StartingCash: 100000,
  AgeDays: 544.54,
  Alpha: 0.1137,
  Beta: 0.0741,
  C2Star: null,
  AnnReturn: 0.5414,
  Correlation_SP500: 0.0647,
  DailyMaxLevMax: 3.31,
  DailyMaxLevMean: 1.00381,
  DeltaEquity30Days: -0.1,
  DeltaEquity45Days: 0.06,
  DeltaEquity90Days: -3.99,
  DeltaEquity180Days: 4.86,
  MaxDrawdownPcnt: 11.39,
  NumTrades: 644,
  PcntMonthsProfitable: 68.4,
  ProfitFactor: 1.398,
  Sharpe: 1.97,
  Sortino: 3.3,
  Calmar: 7.129,
  MaxDrawdownPcnt180days: 6.31615,
  MaxDrawdownPcnt365days: 8.88536,
  TradeDaysAgo: 0.7,
  ShortOptionsCovered: 100,
  OptionPercent: 0,
  MaxWorstLossPercentEquity015: 0.79,
  MaxWorstLossPercentEquity030: 1.31,
  MaxWorstLossPercentEquity045: 1.31,
  MaxWorstLossPercentEquity060: 1.31,
  MaxWorstLossPercentEquity090: 1.71,
  MaxWorstLossPercentEquity180: 51.94,
  scoreRisk: 51.66277777777778,
  scoreBehavior: 92.05222222222221,
  scorePr

In [18]:
// Select some data and visualize it.
let uidata = data
            .map(i => ( 
                  {  
                      Score: i.score.toFixed(2), 
                      StrategyId: i.StrategyId,  
                      StrategyName: i.StrategyName, 
                      scoreRisk: i.scoreRisk.toFixed(2),
                      scoreBehavior: i.scoreBehavior.toFixed(2),
                      scoreProfitability: i.scoreProfitability.toFixed(2)                      
                  })
                )            ;
console.table(uidata.slice(0,20));

┌─────────┬──────────┬────────────┬────────────────────────────┬───────────┬───────────────┬────────────────────┐
│ (index) │  Score   │ StrategyId │        StrategyName        │ scoreRisk │ scoreBehavior │ scoreProfitability │
├─────────┼──────────┼────────────┼────────────────────────────┼───────────┼───────────────┼────────────────────┤
│    0    │ '161.76' │ 128743489  │  'JWB Stock and ETF ORB'   │  '51.66'  │    '92.05'    │      '18.05'       │
│    1    │ '161.01' │ 126454200  │  'GardCap Discretionary'   │  '55.83'  │    '99.83'    │       '5.35'       │
│    2    │ '160.07' │ 126548162  │     'NASDAQ Momentum'      │  '54.64'  │    '99.26'    │       '6.17'       │
│    3    │ '156.22' │ 117734561  │       'TQQQ Aspire'        │  '44.55'  │    '94.53'    │      '17.14'       │
│    4    │ '154.75' │ 124696549  │     '4Timing Trend ML'     │  '51.46'  │    '99.49'    │       '3.79'       │
│    5    │ '152.95' │ 129134794  │         'JC Alpha'         │  '51.07'  │    '94.63' 

In [19]:
// A little nicer presentation in a simple HTML format genereted by the C2Utils.htmlTable function:
C2Utils.htmlTable(uidata.slice(0,20));

Score StrategyId StrategyName scoreRisk scoreBehavior scoreProfitability 161.76 128743489 JWB Stock and ETF ORB 51.66 92.05 18.05 161.01 126454200 GardCap Discretionary 55.83 99.83 5.35 160.07 126548162 NASDAQ Momentum 54.64 99.26 6.17 156.22 117734561 TQQQ Aspire 44.55 94.53 17.14 154.75 124696549 4Timing Trend ML 51.46 99.49 3.79 152.95 129134794 JC Alpha 51.07 94.63 7.25 152.15 125765564 Beating billionaires 48.35 96.62 7.18 150.57 126054331 CkNN Algo V 33.38 97.54 19.65 149.99 126444124 Saivers US ETF 37.56 96.23 16.20 148.13 125587405 stock star 49.32 82.44 16.37 148.11 123231599 SUPER15H Hedged Stocks 33.49 105.92 8.70 147.73 97088697 4Timing Machine Learning 46.75 99.51 1.47 147.28 125443378 PatternProfits 38.28 99.57 9.43 147.06 98995198 Portfolio Trader PRO 41.88 100.85 4.33 146.88 106004078 BlackAce 47.02 94.11 5.76 146.87 128415506 ALPS 1 39.29 87.58 20.00 145.66 77477692 The Vegan Growth Port 36.72 103.98 4.96 144.73 130576078 10 Bagger Investing 41.37 83.35 20.00 143.99 130018805 Stocks for Div yield 52.31 86.64 5.04 142.50 125624499 dow m 52.12 70.38 20.00

In [20]:
// Or this one:
C2Utils.htmlTable(uidata.slice(0,20),true);

Chart Score StrategyId StrategyName scoreRisk scoreBehavior scoreProfitability 161.76 128743489 JWB Stock and ETF ORB 51.66 92.05 18.05 161.01 126454200 GardCap Discretionary 55.83 99.83 5.35 160.07 126548162 NASDAQ Momentum 54.64 99.26 6.17 156.22 117734561 TQQQ Aspire 44.55 94.53 17.14 154.75 124696549 4Timing Trend ML 51.46 99.49 3.79 152.95 129134794 JC Alpha 51.07 94.63 7.25 152.15 125765564 Beating billionaires 48.35 96.62 7.18 150.57 126054331 CkNN Algo V 33.38 97.54 19.65 149.99 126444124 Saivers US ETF 37.56 96.23 16.20 148.13 125587405 stock star 49.32 82.44 16.37 148.11 123231599 SUPER15H Hedged Stocks 33.49 105.92 8.70 147.73 97088697 4Timing Machine Learning 46.75 99.51 1.47 147.28 125443378 PatternProfits 38.28 99.57 9.43 147.06 98995198 Portfolio Trader PRO 41.88 100.85 4.33 146.88 106004078 BlackAce 47.02 94.11 5.76 146.87 128415506 ALPS 1 39.29 87.58 20.00 145.66 77477692 The Vegan Growth Port 36.72 103.98 4.96 144.73 130576078 10 Bagger Investing 41.37 83.35 20.00 143.99 130018805 Stocks for Div yield 52.31 86.64 5.04 142.50 125624499 dow m 52.12 70.38 20.00